# Setup

In [ ]:
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 7.3 MB/s 
     |████████████████████████████████| 4.4 MB 28.2 MB/s 
     |████████████████████████████████| 596 kB 66.0 MB/s 
     |████████████████████████████████| 6.6 MB 65.6 MB/s 
     |████████████████████████████████| 101 kB 12.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Body

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/euphemism/train.csv")

In [ ]:
replacements = {}
# replacements['pass away'] = 'die'
replacements['weed'] = 'marijuana'
replacements['overweight'] = 'obese'
replacements['aging'] = 'growing old'
replacements['troubled'] = 'problematic'
replacements['let go of'] = 'fire'
replacements['exterminate'] = 'kill'
replacements['disabled'] = 'hobbled'
replacements['late'] = 'dead'
replacements['sober'] = 'not drunk'
# replacements['mixed up'] = 'confused'     # Probably wrong; I have no clue about this
replacements['economical'] = 'prudent'
replacements['collateral damage'] = 'deaths'
replacements['regime change'] = 'government overthrow'
replacements['dismissed'] = 'fired'
replacements['perish'] = 'die'
replacements['demise'] = 'death'
replacements['special needs'] = 'handicapped'
replacements['intoxicated'] = 'drunk'
replacements['well off'] = 'rich'
replacements['slim'] = 'slender'
replacements['same-sex'] = 'homosexual'
replacements['underdeveloped'] = 'poor'
# replacements['lay off'] = 'fire'
replacements['a certain age'] = 'an old age'
replacements['disadvantaged'] = 'poor'
replacements['wealthy'] = 'billionaires'
replacements['custodian'] = 'security guard'

In [ ]:
from bert_score import score
def get_bertscore(cands, refs):
    P,R,F = score(cands, refs, lang="en", rescale_with_baseline=True)
    return F

In [ ]:
# Idea: Replace word with synonym and get bertscore.
# e.g. "He likes <weed>." --> "He likes marijuana."
# Get BERTScore between new sentence and old sentence.
# Do this for all occurrences of the word. Sort all the BERTScores. Predict the top half of the scores as 1 and bottom half of the scores as 0.
# All misclassified rows are considered "possibly mislabelled".

gtruth, preds = [], []
candidate_sents = []
for word in replacements:
    curr_sents, curr_labels = [], []
    for i,l in zip(df.utterance.tolist(), df.label.tolist()):
        if f"<{word}>" in i: 
            curr_sents.append(i)
            curr_labels.append(l)
    orig_sents = [i.replace(f"<{word}>", f"{word}") for i in curr_sents]
    new_sents = [i.replace(f"<{word}>", f"{replacements[word]}") for i in curr_sents]
    out = get_bertscore(new_sents, orig_sents)
    vals = []
    for i in range(len(curr_labels)):
        vals.append((out[i].item(), curr_labels[i], i))
    print(word)
    print(sorted(vals))
    vals_sorted = sorted(vals)
    halfway_point = len(vals_sorted)//2
    for i in range(len(vals_sorted)):
        gtruth.append(vals_sorted[i][1])
        if i < halfway_point: preds.append(0)
        else: preds.append(1)
        if gtruth[-1] != preds[-1]:
            candidate_sents.append((word, replacements[word], curr_sents[vals_sorted[i][2]], vals_sorted[i][1]))
    print(len(candidate_sents), '\n')

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


weed
[(0.8875076770782471, 0, 21), (0.9268372058868408, 0, 15), (0.941048800945282, 0, 1), (0.9469472765922546, 0, 17), (0.9482998847961426, 0, 2), (0.9518586993217468, 0, 30), (0.9531541466712952, 0, 16), (0.9620869755744934, 0, 6), (0.9627523422241211, 0, 23), (0.9648572206497192, 0, 0), (0.9654780626296997, 0, 26), (0.967638373374939, 0, 12), (0.9686861634254456, 0, 7), (0.973893940448761, 0, 5), (0.9791638255119324, 0, 8), (0.9817492961883545, 0, 11), (0.9830175042152405, 1, 31), (0.984182596206665, 1, 20), (0.9862167835235596, 1, 22), (0.9863718152046204, 1, 18), (0.9882969260215759, 1, 27), (0.9897271990776062, 1, 10), (0.9918003082275391, 1, 9), (0.9922448992729187, 1, 3), (0.9925196766853333, 1, 24), (0.9940700531005859, 1, 28), (0.9942957162857056, 1, 25), (0.9949727654457092, 1, 13), (0.9956896305084229, 1, 4), (0.9963610172271729, 1, 14), (0.997857391834259, 0, 19), (0.9983044862747192, 1, 29)]
1 



Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


overweight
[(0.9501303434371948, 0, 1), (0.9748644232749939, 0, 13), (0.9753952026367188, 0, 27), (0.9820395708084106, 1, 11), (0.982391357421875, 1, 14), (0.9831940531730652, 1, 10), (0.9836034178733826, 0, 29), (0.9841977953910828, 0, 3), (0.9845774173736572, 1, 22), (0.984659731388092, 0, 26), (0.9863220453262329, 0, 23), (0.9866060018539429, 1, 20), (0.9879041910171509, 1, 7), (0.9897078275680542, 0, 12), (0.9906383752822876, 1, 0), (0.991067111492157, 1, 18), (0.9922021627426147, 0, 6), (0.9922237396240234, 0, 17), (0.9928911924362183, 0, 19), (0.9930596351623535, 0, 21), (0.9930635690689087, 0, 25), (0.9933277368545532, 0, 31), (0.9936286211013794, 1, 32), (0.9937356114387512, 1, 5), (0.9943847060203552, 1, 9), (0.9945927262306213, 1, 4), (0.9948462843894958, 0, 16), (0.9949554204940796, 1, 30), (0.995079755783081, 1, 24), (0.9951542615890503, 0, 8), (0.9951987862586975, 0, 33), (0.9953545331954956, 1, 28), (0.9955815672874451, 0, 15), (0.997390866279602, 1, 2)]
18 



Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

disadvantaged
[(0.948827862739563, 1, 2), (0.9567902684211731, 0, 19), (0.959494411945343, 0, 10), (0.9607919454574585, 0, 3), (0.9644916653633118, 0, 5), (0.9671781659126282, 0, 11), (0.9742749929428101, 0, 17), (0.9781661033630371, 0, 24), (0.9785923957824707, 0, 13), (0.9786263108253479, 1, 7), (0.9817863702774048, 0, 8), (0.9822433590888977, 1, 25), (0.9829105138778687, 0, 15), (0.9848557114601135, 0, 14), (0.9859610795974731, 1, 18), (0.9874747395515442, 1, 23), (0.9887557029724121, 1, 6), (0.9888139367103577, 1, 20), (0.9896145462989807, 1, 1), (0.9905815124511719, 0, 4), (0.9906521439552307, 1, 16), (0.991081953048706, 1, 0), (0.992344856262207, 1, 22), (0.9929890036582947, 1, 12), (0.9936935901641846, 0, 21), (0.9957235455513, 1, 9)]
196 



Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


wealthy
[(0.9204820394515991, 1, 6), (0.9507642984390259, 0, 4), (0.9527812004089355, 1, 2), (0.9606810808181763, 0, 1), (0.9631941914558411, 1, 0), (0.9784190058708191, 1, 7), (0.979624330997467, 0, 5), (0.9829009771347046, 0, 3)]
200 



Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


custodian
[(0.9489575028419495, 1, 5), (0.9532763361930847, 0, 3), (0.9653382301330566, 0, 8), (0.9732412695884705, 0, 4), (0.9732585549354553, 1, 2), (0.9753454327583313, 0, 0), (0.9756985902786255, 1, 7), (0.976039707660675, 1, 1), (0.9857181310653687, 1, 6)]
202 



In [ ]:
candidate_sents

[('weed',
  'marijuana',
  'They provide less uncompensated care for the poor and the elderly. "Because of the sovereignty granted to the AMA back in the 1930? s @ @ @ @ @ @ @ @ @ @ the privilege should be withdrawn from the AMA. Because for profit hospitals can raise (bilk investors out of ) capital easier than non profit thru issuing stock only means that for profit hospitals are more likely to skim the patient populations ( the practice of only treating patitents with good insurance and leaving the rest to the wolves) in theory, it has been said that " competition " would " <weed> out" less efficient systems of care, but in reality, those claimed outcomes by the bantering jackyls of the insurance lobby never seem to materialize. The U.S is the only western nation without some kind of universal health care. ',
  0),
 ('overweight',
  'obese',
  'BMI is a measurement based on height and weight. Between 25 and 30 is considered <overweight> @ @ @ @ @ @ @ @ @ @ officials have found.',
  

In [ ]:
euphs_final, replacements_final, sents_final, labels_final = [], [], [], []
for c in candidate_sents:
    euphs_final.append(c[0])
    replacements_final.append(c[1])
    sents_final.append(c[2])
    labels_final.append(c[3])
df_final = pd.DataFrame({"euph":euphs_final, "replacement":replacements_final, "utterance":sents_final, "label":labels_final})
df_final.to_csv("/content/drive/MyDrive/Colab Notebooks/euphemism/data/candidate_replacements.csv", index=False)

# Scratchwork (might be useful for understanding)

In [ ]:
sents, labels = [], []
for i,l in zip(df.utterance.tolist(), df.label.tolist()):
    if "<wealthy>" in i: 
        sents.append(i)
        labels.append(l)

In [ ]:
orig_sents = [i.replace("<wealthy>", "wealthy") for i in sents]
new_sents = [i.replace("<wealthy>", "billionaires") for i in sents]
out = get_bertscore(new_sents, orig_sents)

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
vals = []
for i in range(len(labels)):
    print(i, out[i].item(), labels[i])
    vals.append((out[i].item(), labels[i], i))

0 0.9631941914558411 1
1 0.9606810808181763 0
2 0.9527812004089355 1
3 0.9829009771347046 0
4 0.9507642984390259 0
5 0.979624330997467 0
6 0.9204820394515991 1
7 0.9784190058708191 1


In [ ]:
labels[3], sents[3]

(0,
 'We all know in large part Republicans are the enemies of economic or social justice. This is not the GOP of a generation ago @ @ @ @ @ @ @ @ @ @ not benefit the <wealthy>. They are bought and paid for by both corporate America and other wealthy global corporations.')

In [ ]:
sorted(vals)

[(0.9204820394515991, 1, 6),
 (0.9507642984390259, 0, 4),
 (0.9527812004089355, 1, 2),
 (0.9606810808181763, 0, 1),
 (0.9631941914558411, 1, 0),
 (0.9784190058708191, 1, 7),
 (0.979624330997467, 0, 5),
 (0.9829009771347046, 0, 3)]